# Ch4. SOLID 원칙
- 참고 블로그: [잔재미코딩](https://www.fun-coding.org/PL&OOP2-1.html)
- 참고 블로그: [doorbw](https://doorbw.tistory.com/240)
- [SOLID Design Principles with Python Examples](https://www.linkedin.com/pulse/solid-design-principles-python-examples-hiral-amodia)

## 1. SRP

### 예제1
- 책임 분산

In [16]:
class ScoreAndCourse(object):
    """2개 이상의 책임을 가진 클래스"""

    def __init__(self):
        scores = {}
        courses = {}

    def get_score(self, student_name, course):
        """학생 성적 관리"""
        pass

    def get_courses(self, student_name):
        """ 학생이 수강하는 강의 관리"""
        pass

- 하나의 클래스에 너무 많은 책임이 있다.

In [17]:
class ScoreManager(object):
    """SRP원칙이 적용된 클래스"""

    def __init__(self):
        scores = {}

    def get_score(self, student_name, course):
        """학생 성적 관리"""
        pass

class CourseManager(object):
    """SRP원칙이 적용된 클래스"""

    def __init__(self):
        courses = {}

    def get_courses(self, student_name):
        """ 학생이 수강하는 강의 관리"""
        pass

## 2. OPC
- 개방폐쇄원칙

### 예제
- 여러 도형에 대한(확장) 넓이 구하기

In [18]:
class AreaCalculator(object):
    """OPC를 적용하지 않은 클래스"""
    def __init__(self, shapes):
        self.shapes = shapes

    def total_area(self):
        total = 0
        for shape in self.shapes:
            total += shape.width * shape.height 
        return total

class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height

class Circle:
    def __init__(self, radius):
        self.radius = radius

In [19]:
shapes = [Rectangle(3, 4), Rectangle(1,6)]
calculator = AreaCalculator(shapes)

print(calculator.total_area())

18


> 다른 넓이 계산 방식의 도형에 대한 확장이 어렵고 AreaCalculator() 클래스를 수정해야 함

In [20]:
class AreaCalculator(object):
    """OPC가 적용되어 수정이 필요없는 클래스"""
    def __init__(self, shapes):
        self.shapes = shapes

    def total_area(self):
        total = 0
        for shape in self.shapes:
            total += shape.area()
        return total

class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def area(self):
        return self.width * self.height

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    def area(self):
        return 3.14 * self.radius ** 2


class Triangle:
    """ 확장이 쉬운 클래스"""
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def area(self):
        return self.width * self.height * 1/2

In [21]:
shapes = [Rectangle(3, 4), Rectangle(1,6) ,Circle(5), Triangle(2, 4)]
calculator = AreaCalculator(shapes)

print(calculator.total_area())

100.5


## 3. LSP
- 리스코프 치환 법칙

### 예제1
- 재정의한 메서드가 다른 타입을 사용

In [22]:
class Event:
    """LSP가 지켜지지 않은 클래스"""

    def meets_condition(self, event_data: dict) -> bool:
        return False


class LoginEvent(Event):
    """ 재정의한 클래스 타입 : LIST"""
    def meets_condition(self, event_data: list) -> bool:
        return bool(event_data)

In [23]:
class Event:
    """Super class: Event class"""
    def __init__(self, event_data: dict):
        self.event_data = event_data
        
    def meet_condition(self, event_data: dict) -> bool:
        return False
    
class LoginEvent(Event):
    """Sub class: LoginEvent class"""
    def meet_condition(self, event_data: dict) -> bool:
        return event_data['before']["session"] == 0 and event_data['after']["session"] == 1

In [24]:
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data

    @staticmethod
    def meets_condition(event_data: dict):
        return False

    @staticmethod
    def meets_condition_pre(event_data: dict):
        """Precondition of the contract of this interface.

        Validate that the ``event_data`` parameter is properly formed.
        """
        assert isinstance(event_data, dict), f"{event_data!r} is not a dict"
        for moment in ("before", "after"):
            assert moment in event_data, f"{moment} not in {event_data}"
            assert isinstance(event_data[moment], dict)
            
class LoginEvent(Event):
    """Sub class: LoginEvent class"""
    @staticmethod
    def meets_condition(event_data: dict) -> bool:
#         assert "session" in event_data["before"] and "session" in event_data["after"]
        return event_data['before']["session"] == 0 and event_data['after']["session"] == 1
 
        
class UnknownEvent(Event):
    def meet_condition(self, event_data: dict) -> bool:
        return True
    
class TransactionEvent(Event):
    """Represents a transaction that has just occurred on the system."""

    @staticmethod
    def meets_condition(event_data: dict):
        return event_data["after"].get("transaction") is not None

class SystemMonitor:
    """Identify events that occurred in the system"""
    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        Event.meets_condition_pre(self.event_data)
        event_cls = next(
            (
                event_cls
                for event_cls in Event.__subclasses__()
                if event_cls.meets_condition(self.event_data)
            ),
            UnknownEvent,
        )
        return event_cls(self.event_data)


In [25]:
l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
l1.identify_event().__class__.__name__
    # 'LoginEvent'

l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
l2.identify_event().__class__.__name__
    # 'LogoutEvent'

l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
l3.identify_event().__class__.__name__
    # 'UnknownEvent'

l4 = SystemMonitor({"before": {}, "after": {"transaction": "Tx001"}})
l4.identify_event().__class__.__name__
    # 'TransactionEvent'

KeyError: 'session'

### 예제2

In [26]:
class Car():
    """ super type class"""
    def __init__(self, type):
        self.type = type

class PetrolCar(Car):
    """sub type class"""
    def __init__(self, type):
        self.type = type

car = Car('SUV')
car.properties = {"Color": "Red", "Gear": "Auto", "Capacity": 6}

petrol_car = PetrolCar("Sedan")
petrol_car.properties = ("Blue", "Manual", 4)

cars = [car, petrol_car]

def find_red_cars(cars):
    red_cars = 0
    for car in cars:
        if car.properties['Color'] == "Red":
            red_cars += 1

    print(f'Number of Red Cars = {red_cars}')

find_red_cars(cars)

TypeError: tuple indices must be integers or slices, not str

In [27]:
class Car():
  def __init__(self, type):
    self.type = type
    self.car_properties = {}
  
  def set_properties(self, color, gear, capacity):
    self.car_properties = {"Color": color, "Gear": gear, "Capacity": capacity}

  def get_properties(self):
    return self.car_properties

class PetrolCar(Car):
  def __init__(self, type):
    self.type = type
    self.car_properties = {}

car = Car("SUV")
car.set_properties("Red", "Auto", 6)

petrol_car = PetrolCar("Sedan")
petrol_car.set_properties("Blue", "Manual", 4)

cars = [car, petrol_car]

def find_red_cars(cars):
  red_cars = 0
  for car in cars:
    if car.get_properties()['Color'] == "Red":
      red_cars += 1
  print(f'Number of Red Cars = {red_cars}')

find_red_cars(cars)

Number of Red Cars = 1


## 4. ISP
- 인터페이스 분리 원칙

### 예제1
- 포유류
  - 사람
  - 고래

- 하나의 추상화 클래스(Mammals)에 2가지의 메서드가 존재

In [28]:
from abc import ABC, abstractmethod

class Mammals(ABC):
    """LSP가 적용안된 클래스"""
    @abstractmethod
    def swim() -> bool:
        print("Can Swim") 

    @abstractmethod
    def walk() -> bool:
        print("Can Walk") 

class Human(Mammals):
    def swim():
        return print("Humans can swim") 

    def walk():
        return print("Humans can walk") 

class Whale(Mammals):
    def swim():
        return print("Whales can swim") 

In [29]:
Human.swim()
Human.walk()

Whale.swim()
Whale.walk() # 잘못된 예

Humans can swim
Humans can walk
Whales can swim
Can Walk


- 올바른 계층구조를 갖게 인터페이스 분리
- 클라이언트는 특별한 주의를 기울이지 않고도 하위 타입을 사용할 수 있음

In [30]:
class Walker(ABC):
  @abstractmethod
  def walk() -> bool:
    return print("Can Walk") 

class Swimmer(ABC):
  @abstractmethod
  def swim() -> bool:
    return print("Can Swim") 

class Human(Walker, Swimmer):
  def walk():
    return print("Humans can walk") 
  def swim():
    return print("Humans can swim") 

class Whale(Swimmer):
  def swim():
    return print("Whales can swim") 

if __name__ == "__main__":
  Human.walk()
  Human.swim()

  Whale.swim()
  Whale.walk()

Humans can walk
Humans can swim
Whales can swim


AttributeError: type object 'Whale' has no attribute 'walk'

### 예제2
- ISP를 지키지 못함
  - 처리하는 데이터와 무관하게 두 가ㄷ지 함수를 모두 구현

In [31]:
from abc import *
 
class EventParser(metaclass=ABCMeta):
    """Interface: EventParser class"""
    @abstractmethod
    def from_json(self, event_data):
        pass
    
    @abstractmethod
    def from_xml(self, event_data):
        pass    


In [32]:
from abc import *
 
class JsonEventParser(metaclass=ABCMeta):
    """Interface: JsonEventParser class"""
    @abstractmethod
    def from_json(self, event_data):
        pass    
    
class XmlEventParser(metaclass=ABCMeta):
    """Interface: XmlEventParser class"""
    @abstractmethod
    def from_xml(self, event_data):
        pass    


## 5. DIP
- 의존성 역전 법칙

### 예제1
- EventStreamer 가 syslog를 직접참조
- syslog 의 메서드 send()가 변경되면 EventStreamer도 변경해야 할 가능성이 생김

In [33]:
from abc import *
 
class EventStreamer():
    """ A: 고수준 모듈"""
    def __init__(self, parsed_data: str, client: Syslog):
        self.parsed_data = parsed_data
        assert client is Syslog, "Client is not Syslog"
        self.client = client
        
    def stream(self):
        self.client.send(self.parsed_data)    
        
class Syslog():
    """ B: 저수준 모듈"""

    def send(data: str):
        print(f"Syslog send: {data}")
        pass
    
class OtherClient():
    def send(data: str):
        print(f"OtherClient send: {data}")
        pass
 
 
streamer1 = EventStreamer("for Syslog data!", Syslog)
streamer1.stream()
streamer2 = EventStreamer("for OtherClient data!", OtherClient)
streamer2.stream()

NameError: name 'Syslog' is not defined

- 새로운 인터페이스를 만들어 의존성을 역전시킴

In [34]:
from abc import *
 
class EventStreamer():
    """ A: 고수준 모듈"""
    def __init__(self, parsed_data: str, client):
        self.parsed_data = parsed_data
        assert client in DataTargetClient.__subclasses__(), "Client is not DataTargetClient"
        self.client = client
        
    def stream(self):
        self.client.send(self.parsed_data)
 
class DataTargetClient(metaclass=ABCMeta):
    """Interface: DataTargetClient class"""
    @abstractmethod
    def send(self, data: str):
        pass            
        
class Syslog(DataTargetClient):
    """ B: 저수준 모듈"""
    def send(data: str):
        print(f"Syslog send: {data}")
        pass
    
class OtherClient(DataTargetClient):
    def send(data: str):
        print(f"OtherClient send: {data}")
        pass
 
 
streamer1 = EventStreamer("for Syslog data!", Syslog)
streamer1.stream()
streamer2 = EventStreamer("for OtherClient data!", OtherClient)
streamer2.stream()

Syslog send: for Syslog data!
OtherClient send: for OtherClient data!


In [3]:
from collections.abc import Sequence
my_dict = {'a': 1}
print(isinstance("abc", Sequence))
print(isinstance(my_dict, Sequence))

True
False
